# Introduction to Machine Learning - Task 3

Group name: Cbbayes

Team members: mcolomer (mcolomer@student.ethz.ch), pratsink (pratsink@student.ethz.ch) and scastro (scastro@student.ethz.ch)

Spring 2021

## Libraries

In [48]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics

## Preprocessing: one-hot encoding of the sequences

In [49]:
# Indicate the path to the data file
path = "../data/"

train_data = pd.read_csv(path+"train.csv")
test_data = pd.read_csv(path+"test.csv")

aminoacids = ["R", "H", "K","D","E", "S","T","N","Q","C","U",
             "G","P","A","I","L","M","F","W","Y","V"]

charge = [1,1,1,-1,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
polarity=[1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0]
special=[0,0,0,0,0,0,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0]


#Preprocessing training dataset

X_train = train_data["Sequence"].str.split("", n = 4, expand = True)

def map_charge(aa, charge, aminoacids):
    i = aminoacids.index(aa)
    return charge[i]

def map_polarity(aa, polarity, aminoacids):
    i = aminoacids.index(aa)
    return polarity[i]

for i in [1,2,3,4]:
    train_data["pos"+str(i)] = X_train[i]
    train_data = pd.concat([train_data,pd.get_dummies(train_data["pos"+str(i)], prefix="pos"+str(i))],axis=1)
    train_data["charge"+str(i)] = train_data["pos"+str(i)].apply(lambda aa: map_charge(aa, charge, aminoacids))
    train_data["polarity"+str(i)] = train_data["pos"+str(i)].apply(lambda aa: map_polarity(aa, polarity, aminoacids))
    train_data = train_data.drop(columns=["pos"+str(i)])
    
X_train = train_data.iloc[:, 2:90].values 
y_train = train_data.iloc[:,1].values 

#Preprocessing test dataset

X_test = test_data["Sequence"].str.split("", n = 4, expand = True)


for i in [1,2,3,4]:
    test_data["pos"+str(i)] = X_test[i]
    test_data = pd.concat([test_data,pd.get_dummies(test_data["pos"+str(i)], prefix="pos"+str(i))],axis=1)
    test_data["charge"+str(i)] = test_data["pos"+str(i)].apply(lambda aa: map_charge(aa, charge, aminoacids))
    test_data["polarity"+str(i)] = test_data["pos"+str(i)].apply(lambda aa: map_polarity(aa, polarity, aminoacids))
    test_data = test_data.drop(columns=["pos"+str(i)])
    
X_test = test_data.iloc[:, 1:89].values 


In [75]:
train_data

,Sequence,Active,pos1_A,pos1_C,pos1_D,pos1_E,pos1_F,pos1_G,pos1_H,pos1_I,...,pos4_P,pos4_Q,pos4_R,pos4_S,pos4_T,pos4_V,pos4_W,pos4_Y,charge4,polarity4
0,DKWL,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,FCHN,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,KDQP,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
3,FNWI,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,NKRM,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111995,GSME,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,-1,1
111996,DLPT,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
111997,SGHC,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
111998,KIGT,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1


In [50]:
print(len(charge))
print(len(polarity))
print(len(special))

21
21
21


## Classification

Class are quite unbalanced:

In [33]:
print(len(y_train))
print(sum(y_train))

112000
4213


In [73]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, make_scorer

# Create the parameter grid based on the results of random search 
param_grid = {
    'n_estimators': [250]
}
# Create a based model
rf = RandomForestClassifier(class_weight='balanced')
# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2, scoring="f1")


grid_search.fit(X_train, y_train)
grid_search.best_params_


Fitting 3 folds for each of 1 candidates, totalling 3 fits


{'n_estimators': 300}

In [92]:
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingClassifier



clf = HistGradientBoostingClassifier(scoring = 'f1',
                                     random_state = 123).fit(X_train, y_train)
    
## Print the testing and traing score. Trainig score is estimated for the probability estimates not the labels.
print(clf.validation_score_[np.size(clf.validation_score_) - 1],
      " ",
      metrics.f1_score(y_train,
                            clf.predict(X_train),
                            average='micro'))


y_test = clf.predict(X_test)


0.8388746803069053   0.9895625


In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn.ensemble import HistGradientBoostingClassifier




In [76]:
scores = [x[1] for x in grid_search.grid_scores_]

AttributeError: 'GridSearchCV' object has no attribute 'grid_scores_'

In [77]:
grid_search.scorer_

make_scorer(f1_score, average=binary)

In [78]:
grid_search.best_score_

0.8095144633851081

In [84]:
grid_search.best_score_

0.4132588950395086

In [64]:
def fit_RFC(X_train, y_train, X_test):
    """Random forest for predicting
    the regression features"""
    clf = RandomForestClassifier(n_estimators=250, class_weight='balanced')
    clf.fit(X_train, y_train)
    predicted = clf.predict(X_test)
    return predicted

In [80]:
from sklearn.ensemble import GradientBoostingClassifier

def fit_RFC(X_train, y_train, X_test):
    """Random forest for predicting
    the regression features"""
    clf = RandomForestClassifier(n_estimators=250, class_weight='balanced')
    clf.fit(X_train, y_train)
    predicted = clf.predict(X_test)
    return predicted

In [ ]:
rf_random.best_params_

In [66]:
y_test = fit_RFC(X_train, y_train, X_test)

In [68]:
print(len(y_test))
print(sum(y_test))

48000
1519


In [93]:
pd.DataFrame(y_test).to_csv('output_HBGC_balanced_charge_polarity.csv', index = False, header = False)             ## Save the results dataframe to a csv